# Imports

In [ ]:
import numpy as np
import torch
import os
import matplotlib.pyplot as plt

# import FaderNetworks Directory
import sys
sys.path.append('../FaderNetworks')

from src.logger import create_logger
from src.loader import load_images, DataSampler
from src.utils import bool_flag
from src.loader import AVAILABLE_ATTR

# Classification

In [1]:
def sig(x):
    return 1/(1 + np.exp(-x))

# Use classifier to classify counterfactual interpolations
def classifications(model, interpolations):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    mod = torch.load(model).eval()
    interps = torch.load(interpolations).to(device)
    outputs = []
    for x in interps:
        outputs.append(mod(x))
    # Take the second value in each pair
    new_outputs = []
    for tensor1 in outputs:
        tens1 = []
        for tensor2 in tensor1:
            tens2 = []
            for x in range(0,80,2):
                tens2.append(sig(tensor2[x+1].cpu().detach().numpy()))
            tens1.append(tens2)
        new_outputs.append(tens1)
    return new_outputs


# Get the classification outputs for the interpolations for one single attribute without including the first 2 images
def isolate_attribute(outputs, attribute):
    attribute_classifications = []
    
    for i in outputs:
        face = []
        for x in i:
            face.append(x[AVAILABLE_ATTR.index(attribute)]) #not including the first 2 images
        attribute_classifications.append(face[2:])
    return attribute_classifications

# Isolate a single attribute along with the original images
def isolate_attribute_with_orig(outputs, attribute):
    attribute_classifications = []
    for i in outputs:
        face = []
        for x in i:
            face.append(x[AVAILABLE_ATTR.index(attribute)]) #including the first 2 images
        attribute_classifications.append(face)
    return attribute_classifications

# MSE

In [ ]:
# Calculate MSE error in FaderNetworks
# Compare the classifciation values of the first face to the interpolated face with the most similar value
def error(outputs, attr):
    attribute_classifications = []
    
    # Find the faces most similar to the original in the grid
    for i in outputs:
        face = []
        for x in i:
            face.append(x[AVAILABLE_ATTR.index(attr)])
        attribute_classifications.append(face)
        
    face_index_arr = []
    for face in attribute_classifications:
        orig = face[0]
        diff = 999
        curr = 2
        for i in range(2,12):
            if(abs(face[i]- orig) < diff):
                diff = face[i]- orig
                curr = i
        face_index_arr.append(curr)
        
    # Sum all the error
    sum = 0
    total = 0
    for attribute in AVAILABLE_ATTR:
        attribute_classifications = isolate_attribute_with_orig(outputs, attribute)
        ind = 0
        for face in attribute_classifications:
            sum += (face[0] - face[face_index_arr[ind]])**2
            ind+=1
            total+=1
    return (sum/total) * 100

# Bias Definitions

In [ ]:
def get_attr_boundary(outputs, prot_attr):
    
    attr_classifications = isolate_attribute(outputs, prot_attr)
    boundary = np.average(attr_classifications)
    
    attr_boundary = []
    for face in attr_classifications:
        for i in range(0,10):
            if face[i] >= boundary:
                attr_boundary.append(i)
                break
            if i == 9:
                attr_boundary.append(8)
        return attr_boundary
def co_occurrence_bias(outputs, attribute):
    biases = []
    prot_boundary = get_attr_boundary(outputs, attribute)
    
    for attribute in AVAILABLE_ATTR:
        attribute_classifications = isolate_attribute(outputs, attribute)
        boundary = np.average(attribute_classifications)
        
        g = 0
        g_prime = 0
        ind = 0
        for face in attribute_classifications:
            for i in range(0,prot_boundary[ind]):
                if face[i] > boundary:
                    g_prime+=1
            for i in range(prot_boundary[ind],10):
                if face[i] > boundary:
                    g+=1
            ind+=1
        bias_score = (g / (g + g_prime)) - (1/2)
        biases.append(bias_score)
    return biases

def demographic_parity(outputs, prot_attr):
    biases = []
    g_boundary = get_attr_boundary(outputs, prot_attr)
    
    for attribute in AVAILABLE_ATTR:
        attribute_classifications = isolate_attribute(outputs, attribute)
        exp_sum = 0
        zg = 0.5
        ind = 0
        for face in attribute_classifications:
            for i in range(0,g_boundary[ind]): #g(x) = 0
                exp_sum += face[i]*(-1)
            for i in range(g_boundary[ind],10): #g(x) = 1
                exp_sum += face[i]*(1/zg - 1)
            ind+=1
        biases.append(exp_sum/40)
    return biases

def equal_opportunity(outputs, prot_attr):
    biases = []
    g_boundary = get_attr_boundary(outputs, prot_attr)
    
    for attribute in AVAILABLE_ATTR:
        attribute_classifications = isolate_attribute(outputs, attribute)
        true_boundary = np.percentile(attribute_classifications, 75)
        exp_sum = 0
        ytrue = 1
        ind = 0
        for face in attribute_classifications:
            px = 0
            pg = 0
            for i in range(0,10):
                if face[i] > true_boundary:
                    px+=1
                    if i >= g_boundary[ind]:
                        pg+=1
            px = px/10
            pg = pg/10
            ind = 0
            for i in range(0,g_boundary[ind]):#g(x) = 0
                if(face[i] > true_boundary):
                    exp_sum += face[i]*(-(ytrue)/px)
            for i in range(g_boundary[ind],10):#g(x) = 1
                if(face[i] > true_boundary and pg != 0):
                    exp_sum += face[i]*(ytrue/pg - ytrue/px)
                #exp_sum += face[i]*(ytrue/0.5 - ytrue/px)
            ind+=1
        biases.append(exp_sum/40)
    return biases


def equalized_odds(outputs, prot_attr):
    biases = []
    g_boundary = get_attr_boundary(outputs, prot_attr)
    
    for attribute in AVAILABLE_ATTR:
        attribute_classifications = isolate_attribute(outputs, attribute)
        true_boundary = np.percentile(attribute_classifications, 75)
        
        exp_sum = 0
        zg = 0.5
        ind = 0
        for face in attribute_classifications:
            px = 0
            pg = 0
            for i in range(0,10):
                if face[i] > true_boundary:
                    px+=1
                    if i > 4:
                        pg+=1
            px = px/10
            pg = pg/10
            ind = 0
            for i in range(0,g_boundary[ind]): #g(x) = 0
                if(face[i] < true_boundary):
                    exp_sum += face[i]*(-(1)/(1-px))
            for i in range(g_boundary[ind],10): #g(x) = 1
                if(face[i] < true_boundary and (zg - pg) != 0):
                    exp_sum += face[i]*((1)/(zg - pg) - (1)/(1-px))
            ind+=1
            
        biases.append(exp_sum/40)
    return biases

# Plotting

In [ ]:
def plot_attribute_grid(outputs):
    fig, axs = plt.subplots(10, 3)
    #2, 24, 31 attractive, no_beard, and smiling
    
    smiling_classifications = isolate_attribute(outputs, "Smiling")
    attractive_classifications = isolate_attribute(outputs, "Attractive")
    no_beard_classifications = isolate_attribute(outputs, "No_Beard")
    
    axs[0,0].plot(smiling_classifications[0])
    axs[0, 0].set_title('Smiling Classifications')
    axs[1,0].plot(smiling_classifications[1])
    axs[2,0].plot(smiling_classifications[2])
    axs[3,0].plot(smiling_classifications[3])
    axs[4,0].plot(smiling_classifications[4])
    axs[5,0].plot(smiling_classifications[5])
    axs[6,0].plot(smiling_classifications[6])
    axs[7,0].plot(smiling_classifications[7])
    axs[8,0].plot(smiling_classifications[8])
    axs[9,0].plot(smiling_classifications[9])
    
    axs[0,1].plot(attractive_classifications[0])
    axs[0, 1].set_title('Attracive Classifications')
    axs[1,1].plot(attractive_classifications[1])
    axs[2,1].plot(attractive_classifications[2])
    axs[3,1].plot(attractive_classifications[3])
    axs[4,1].plot(attractive_classifications[4])
    axs[5,1].plot(attractive_classifications[5])
    axs[6,1].plot(attractive_classifications[6])
    axs[7,1].plot(attractive_classifications[7])
    axs[8,1].plot(attractive_classifications[8])
    axs[9,1].plot(attractive_classifications[9])
    
    axs[0,2].plot(no_beard_classifications[0])
    axs[0, 2].set_title('No Beard Classifications')
    axs[1,2].plot(no_beard_classifications[0])
    axs[2,2].plot(no_beard_classifications[0])
    axs[3,2].plot(no_beard_classifications[0])
    axs[4,2].plot(no_beard_classifications[0])
    axs[5,2].plot(no_beard_classifications[0])
    axs[6,2].plot(no_beard_classifications[0])
    axs[7,2].plot(no_beard_classifications[0])
    axs[8,2].plot(no_beard_classifications[0])
    axs[9,2].plot(no_beard_classifications[0])
    
def plot_attribute_bias(biases, title, color):
    x = list(range(40))
    plt.barh(x,biases, tick_label=AVAILABLE_ATTR, color = color)
    plt.title(title)
    plt.grid()

def plot_multiple_bias(bias1, bias2, bias3, bias4):
    width = 0.3
    x = np.arange(40)
    xvals = bias1
    bar1 = plt.barh(x, xvals, width, color = 'r')
    yvals = bias2
    bar2 = plt.barh(x+width, yvals, width, color='g')
    zvals = bias3
    bar3 = plt.barh(x+width*2, zvals, width, color = 'b')
    avals = bias4
    bar4 = plt.barh(x+width*3, avals, width, color = 'y')
    
    plt.ylabel('Scores')
    plt.title("Young Interpolations Bias")
    plt.yticks(x+width, AVAILABLE_ATTR)
    plt.legend( (bar1, bar2, bar3,bar4), ('co-occurence', 'Demographic Parity', 'Equal Opportunity', 'Equalized Odds') )
    plt.show()
    
def plot_grid(bar1, bar2, bar3, bar4, title):
    x = np.arange(40)
    fig, axs = plt.subplots(2, 2)
    fig.suptitle(title)
    
    axs[0, 0].barh(x, bar1, tick_label = AVAILABLE_ATTR)
    axs[0, 0].set_title('Co-Occurance Bias')
    axs[0, 0].grid(which='minor', alpha=0.1)
    axs[0, 0].grid(which='major', alpha=0.2)
    axs[0, 1].barh(x, bar2, color = 'orange', tick_label = AVAILABLE_ATTR)
    axs[0, 1].set_title('Demographic Parity')
    axs[0, 1].grid(which='minor', alpha=0.1)
    axs[0, 1].grid(which='major', alpha=0.2)
    axs[1, 1].barh(x, bar3, color = 'green', tick_label = AVAILABLE_ATTR)
    axs[1, 1].set_title('Equal Opportunity')
    axs[1, 1].grid(which='minor', alpha=0.1)
    axs[1, 1].grid(which='major', alpha=0.2)
    axs[1, 0].barh(x, bar4, color = 'red', tick_label = AVAILABLE_ATTR)
    axs[1, 0].set_title('Equalized Odds')
    axs[1, 0].grid(which='minor', alpha=0.1)
    axs[1, 0].grid(which='major', alpha=0.2)

# Young Plots

In [ ]:
young_outputs = classifications('classifier256.pth', 'young_interpolations.pth')
co_bias = co_occurrence_bias(young_outputs, "Young")
dem_par = demographic_parity(young_outputs, "Young")
eq_opp = equal_opportunity(young_outputs, "Young")
eq_odds = equalized_odds(young_outputs, "Young")
plot_grid(co_bias, dem_par, eq_opp, eq_odds, "Bias on the Young Attribute")
plot_attribute_grid(young_outputs)

# Male Plots

In [ ]:
male_outputs = classifications('classifier256.pth', 'male_interpolations.pth')
co_bias = co_occurrence_bias(male_outputs, "Male")
dem_par = demographic_parity(male_outputs, "Male")
eq_opp = equal_opportunity(male_outputs, "Male")
eq_odds = equalized_odds(male_outputs, "Male")
plot_grid(co_bias, dem_par, eq_opp, eq_odds, "Bias on the Male Attribute")
plot_attribute_grid(male_outputs)

# Eyeglasses Plots

In [ ]:
eyeglasses_outputs = classifications('classifier256.pth', 'eyeglasses_interpolations.pth')
co_bias = co_occurrence_bias(eyeglasses_outputs, "Male")
dem_par = demographic_parity(eyeglasses_outputs, "Male")
eq_opp = equal_opportunity(eyeglasses_outputs, "Male")
eq_odds = equalized_odds(eyeglasses_outputs, "Male")
plot_grid(co_bias, dem_par, eq_opp, eq_odds, "Bias on the Eyeglasses Attribute")
plot_attribute_grid(eyeglasses_outputs)